🔥 Welcome to Day 7: Transfer Learning with Pretrained CNN Models 🔥

🎯 Goals Today:

1. Load model pretrained (misal: ResNet18)
2. Fine-tune model untuk dataset kecil (contoh: CIFAR-10)
3. Train & evaluate performance

🧠 1. Load Pretrained Model: ResNet18

In [ ]:
import torch
import torchvision.models as models

# load pretained ResNet18
resnet18 = models.resnet18(pretrained=True)

# freeze all layer
for param in resnet18.parameters():
    param.requires_grad = False


# change output layer for 10 classes(cifar-10)
import torch.nn as nn
resnet18.fc = nn.Linear(resnet18.fc.in_features, 10)

c:\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\DELL/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:20<00:00, 2.23MB/s]


📦 2. Load CIFAR-10 Dataset

In [8]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# transform (resize to 224x244 for ResNet input)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

# Load CIFAR-10
train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1000)

100%|██████████| 170M/170M [04:15<00:00, 666kB/s]  


🏋️‍♂️ 3. Train Output Layer Only

In [ ]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = resnet18.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18.fc.parameters(), lr=0.001)

#training loop(only 3 epoch for fast)
for epoch in range(3):
    resnet18.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = resnet18(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

print(f"Epoch {epoch+1}, loss: {running_loss/len(train_loader):.4f}")

✅ 4. Evaluate Accuracy

In [ ]:
resnet18.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet18(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

📝 Exercises Day 7:

1. Use ResNet34 Instead of ResNet18
2. Train and Evaluate Model on CIFAR-10
3. (Challenge): Fine-Tune the Whole Model

🧪 Exercise 1: Use ResNet34 Instead of ResNet18

✅ Step-by-step:

In [ ]:
# 1. Load pretrained ResNet34
resnet34 = models.resnet34(pretrained=True)

# 2. Freeze all layers (optional, only train the final layer)
for param in resnet34.parameters():
    param.requires_grad = False

# 3. Modify the output layer for 10 classes (e.g. CIFAR-10)
resnet34.fc = nn.Linear(resnet34.fc.in_features, 10)

# 4. Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet34 = resnet34.to(device)

# 5. Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet34.fc.parameters(), lr=0.001)


🧪 Exercise 2: Train and Evaluate Model on CIFAR-10

In [ ]:
# Training loop
for epoch in range(3):
    resnet34.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = resnet34(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

✅ Evaluate:

In [ ]:
resnet34.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet34(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"✅ Test Accuracy with ResNet34: {100 * correct / total:.2f}%")

🧪 Exercise 3 (Challenge): Fine-Tune the Whole Model

Unfreeze all layers and do full backpropagation:

In [ ]:
# Unfreeze all layers for fine-tuning
for param in resnet34.parameters():
    param.requires_grad = True

# Redefine optimizer to update all parameters
optimizer = optim.Adam(resnet34.parameters(), lr=1e-4)

# Then re-run the training loop above (use fewer epochs if slow)